### 2. Feature Engineering

**Customer Demographics Features**

1. Age = Current_Year - Year_Birth

2. Is_Parent = 1 if (Kidhome + Teenhome) > 0 else 0

3. Total_Kids = Kidhome + Teenhome

4. Marital_Status_Simplified: Group marital status into broader categories:

- "Married" or "Together" → **Partnered**  
- All others (e.g., "Single", "Divorced", "Widow", etc.) → **Single**

**Purchase Behavior Features**

1. Total_Spent = MntWines + MntFruits + MntMeatProducts + MntFishProducts + MntSweetProducts + MntGoldProds

2. Spending_per_Category:
 
    Share of spending per category (example for Wine and Meat):  
    - Wine_Share = MntWines / Total_Spent  
    - Meat_Share = MntMeatProducts / Total_Spent  
    *(Apply similarly for other product categories)*

3. Average_Spend_per_Purchase: Number of times the customer purchased (Num of Deals Purchases + Num of Web Purchases + Num Catalog Purchases + Num Store Purchases) per total amount spent
 
4. Deal_Dependency: Are the customers depended on deals? 
 
    Deal_Rate = NumDealsPurchases / Total_Purchases

**Channel Engagement Features**

1. Total_Purchases: Num of Web Purchases + Num of Catalog Purchases + Num of Store Purchases

2. Web_Engagement:
 
- Web_Purchase_Share = NumWebPurchases / Total_Purchases  
- Web_Visit_to_Purchase_Ratio = NumWebVisitsMonth / (NumWebPurchases + 1)

3. Is_Online_Buyer: if Num of Web Purchases more than Num Store Purchases

**Temporal Features**

1. Customer_Since_Days: Since when the customer is with us? 
 
    Customer_Since_Days = Today - Date of Customer

2. Recent_Activity: Did the customer made any Purchase in less than 30 days?
 
    Is_Active = 1 if Recency < 30 else 0

3. Customer_Lifetime_Spent_per_Day: How much did the consumer spent since he was a customer with us?
 
    Lifetime_Spend_per_Day = Total_Spent / Customer_Since_Days

**Campaign Response Features**

1. Total_Accepted_Campaigns: Sum of all campaign columns

    Total_Accepted_Campaigns = AcceptedCmp1 + AcceptedCmp2 + AcceptedCmp3 + AcceptedCmp4 + AcceptedCmp5

2. Campaign_Response_Rate: Total accepted Campaigns per total number of campaigns

3. Is_Responder: Does the consumer responded to any of our campaigns?
 
    Is_Responder = 1 if Total_Accepted_Campaigns > 0 or Response == 1 else 0

In [51]:
import pandas as pd
from datetime import datetime
from datetime import date
import os
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

# suppress warnings from final output
import warnings
warnings.simplefilter("ignore")

# set up to view all the info of the columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [52]:
#load data
#df = pd.read_csv('/Users/adityaagarwal/Library/CloudStorage/OneDrive-NortheasternUniversity/Jupyter Notebook/Projects/CRM-Analysis-for-Marketing-data/marketing data/marketing_data.csv')
df = pd.read_csv('D:/OneDrive - Northeastern University/Jupyter Notebook/Data Science Projects/CRM-Analysis-for-Marketing-data/marketing data/clean_marketing_data.csv')
df.head()

,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Response,Complain,Country
0,1970,Graduation,Divorced,84835,0,0,2014-06-16,0,189,104,379,111,189,218,1,4,4,6,1,0,0,0,0,0,1,0,SP
1,1961,Graduation,Single,57091,0,0,2014-06-15,0,464,5,64,7,0,37,1,7,3,7,5,0,0,0,0,1,1,0,CA
2,1958,Graduation,Married,67267,0,1,2014-05-13,0,134,11,59,15,2,30,1,3,2,5,2,0,0,0,0,0,0,0,US
3,1967,Graduation,Together,32474,1,1,2014-05-11,0,10,0,1,0,0,0,1,1,0,2,7,0,0,0,0,0,0,0,AUS
4,1989,Graduation,Single,21474,1,0,2014-04-08,0,6,16,24,11,0,34,2,3,1,2,7,1,0,0,0,0,1,0,SP


In [53]:
# Convert Dt_Customer to datetime
df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'], format='mixed')

##### **Customer Demographics Features**

1. Age

In [54]:
df.head()

,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Response,Complain,Country
0,1970,Graduation,Divorced,84835,0,0,2014-06-16,0,189,104,379,111,189,218,1,4,4,6,1,0,0,0,0,0,1,0,SP
1,1961,Graduation,Single,57091,0,0,2014-06-15,0,464,5,64,7,0,37,1,7,3,7,5,0,0,0,0,1,1,0,CA
2,1958,Graduation,Married,67267,0,1,2014-05-13,0,134,11,59,15,2,30,1,3,2,5,2,0,0,0,0,0,0,0,US
3,1967,Graduation,Together,32474,1,1,2014-05-11,0,10,0,1,0,0,0,1,1,0,2,7,0,0,0,0,0,0,0,AUS
4,1989,Graduation,Single,21474,1,0,2014-04-08,0,6,16,24,11,0,34,2,3,1,2,7,1,0,0,0,0,1,0,SP


In [55]:
date.today()

datetime.date(2025, 9, 17)

In [56]:
df['Age'] = datetime.now().year - df['Year_Birth']

2. is_parent

In [57]:
df['Is_Parent'] = ((df['Kidhome'] + df['Teenhome']) > 0).astype(int)
df['Is_Parent'].value_counts().sum

<bound method Series.sum of Is_Parent
1    1572
0     621
Name: count, dtype: int64>

3. Total_Kids

In [58]:
df['Total_Kids'] = df['Kidhome'] + df['Teenhome']

4. Marital Status Simplified

In [59]:
df['Marital_Status_Simplified'] = df['Marital_Status'].replace({
    'Married': 'Partnered',
    'Together': 'Partnered',
    'Single': 'Single',
    'Divorced': 'Single',
    'Widow': 'Single',
    'Alone': 'Single',
    'Absurd': 'Single',
    'YOLO': 'Single'
})

df.drop(columns =['Marital_Status'], inplace = True)
df['Marital_Status_Simplified'].value_counts().sum

<bound method Series.sum of Marital_Status_Simplified
Partnered    1415
Single        771
Other           7
Name: count, dtype: int64>

##### **Purchase Behaviour Features**
1. Total_Spent

In [60]:
spend_cols = ['MntWines', 'MntFruits', 'MntMeatProducts',
              'MntFishProducts', 'MntSweetProducts', 'MntGoldProds']
df['Total_Spent'] = df[spend_cols].sum(axis=1)

2. Spending per Category Share

In [61]:
for col in spend_cols:
    share_col = col + '_Share'
    df[share_col] = df[col] / df['Total_Spent'].replace(0, 1)

df.head()

,Year_Birth,Education,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Response,Complain,Country,Age,Is_Parent,Total_Kids,Marital_Status_Simplified,Total_Spent,MntWines_Share,MntFruits_Share,MntMeatProducts_Share,MntFishProducts_Share,MntSweetProducts_Share,MntGoldProds_Share
0,1970,Graduation,84835,0,0,2014-06-16,0,189,104,379,111,189,218,1,4,4,6,1,0,0,0,0,0,1,0,SP,55,0,0,Single,1190,0.158824,0.087395,0.318487,0.093277,0.158824,0.183193
1,1961,Graduation,57091,0,0,2014-06-15,0,464,5,64,7,0,37,1,7,3,7,5,0,0,0,0,1,1,0,CA,64,0,0,Single,577,0.804159,0.008666,0.110919,0.012132,0.000000,0.064125
2,1958,Graduation,67267,0,1,2014-05-13,0,134,11,59,15,2,30,1,3,2,5,2,0,0,0,0,0,0,0,US,67,1,1,Partnered,251,0.533865,0.043825,0.235060,0.059761,0.007968,0.119522
3,1967,Graduation,32474,1,1,2014-05-11,0,10,0,1,0,0,0,1,1,0,2,7,0,0,0,0,0,0,0,AUS,58,1,2,Partnered,11,0.909091,0.000000,0.090909,0.000000,0.000000,0.000000
4,1989,Graduation,21474,1,0,2014-04-08,0,6,16,24,11,0,34,2,3,1,2,7,1,0,0,0,0,1,0,SP,36,1,1,Single,91,0.065934,0.175824,0.263736,0.120879,0.000000,0.373626


3. Average Spend per Purchase

In [62]:
df['Total_Purchases'] = df[['NumDealsPurchases', 'NumWebPurchases',
                            'NumCatalogPurchases', 'NumStorePurchases']].sum(axis=1)
df['Avg_Spend_per_Purchase'] = df['Total_Spent'] / df['Total_Purchases'].replace(0, 1)

4. Deal Dependency

In [63]:
df['Deal_Rate'] = df['NumDealsPurchases'] / df['Total_Purchases'].replace(0, 1)

#### **Channel Engagement Features:**
1. Web Engagement

In [64]:
df['Web_Purchase_Share'] = df['NumWebPurchases'] / df['Total_Purchases'].replace(0, 1)
df['Web_Visit_to_Purchase_Ratio'] = df['NumWebVisitsMonth'] / (df['NumWebPurchases'] + 1)

2. Is Online Buyer

In [65]:
df['Is_Online_Buyer'] = (df['NumWebPurchases'] > df['NumStorePurchases']).astype(int)

#### **Temporal Features:**
1. Customer Since Days

In [66]:
df['Customer_Since_Days'] = (pd.to_datetime('2024-01-01') - df['Dt_Customer']).dt.days

2. Is Active (based on Recency)

In [67]:
df['Is_Active'] = (df['Recency'] < 30).astype(int)

3. Lifetime Spend per Day

In [68]:
df['Lifetime_Spend_per_Day'] = df['Total_Spent'] / df['Customer_Since_Days'].replace(0, 1)

##### **Campaign Response Features**
1. Total Accepted Campaigns

In [69]:
campaign_cols = ['AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5']
df['Total_Accepted_Campaigns'] = df[campaign_cols].sum(axis=1)
df['Total_Accepted_Campaigns'].value_counts()

Total_Accepted_Campaigns
0    1738
1     319
2      82
3      43
4      11
Name: count, dtype: int64

2. Campaign Response Rate

In [70]:
df['Campaign_Response_Rate'] = df['Total_Accepted_Campaigns'] / 5

3. Is Responder

In [71]:
df['Is_Responder'] = ((df['Total_Accepted_Campaigns'] > 0) | (df['Response'] == 1)).astype(int)
df.Is_Responder.value_counts()

Is_Responder
0    1596
1     597
Name: count, dtype: int64

In [72]:
df.head()

,Year_Birth,Education,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Response,Complain,Country,Age,Is_Parent,Total_Kids,Marital_Status_Simplified,Total_Spent,MntWines_Share,MntFruits_Share,MntMeatProducts_Share,MntFishProducts_Share,MntSweetProducts_Share,MntGoldProds_Share,Total_Purchases,Avg_Spend_per_Purchase,Deal_Rate,Web_Purchase_Share,Web_Visit_to_Purchase_Ratio,Is_Online_Buyer,Customer_Since_Days,Is_Active,Lifetime_Spend_per_Day,Total_Accepted_Campaigns,Campaign_Response_Rate,Is_Responder
0,1970,Graduation,84835,0,0,2014-06-16,0,189,104,379,111,189,218,1,4,4,6,1,0,0,0,0,0,1,0,SP,55,0,0,Single,1190,0.158824,0.087395,0.318487,0.093277,0.158824,0.183193,15,79.333333,0.066667,0.266667,0.200,0,3486,1,0.341365,0,0.0,1
1,1961,Graduation,57091,0,0,2014-06-15,0,464,5,64,7,0,37,1,7,3,7,5,0,0,0,0,1,1,0,CA,64,0,0,Single,577,0.804159,0.008666,0.110919,0.012132,0.000000,0.064125,18,32.055556,0.055556,0.388889,0.625,0,3487,1,0.165472,1,0.2,1
2,1958,Graduation,67267,0,1,2014-05-13,0,134,11,59,15,2,30,1,3,2,5,2,0,0,0,0,0,0,0,US,67,1,1,Partnered,251,0.533865,0.043825,0.235060,0.059761,0.007968,0.119522,11,22.818182,0.090909,0.272727,0.500,0,3520,1,0.071307,0,0.0,0
3,1967,Graduation,32474,1,1,2014-05-11,0,10,0,1,0,0,0,1,1,0,2,7,0,0,0,0,0,0,0,AUS,58,1,2,Partnered,11,0.909091,0.000000,0.090909,0.000000,0.000000,0.000000,4,2.750000,0.250000,0.250000,3.500,0,3522,1,0.003123,0,0.0,0
4,1989,Graduation,21474,1,0,2014-04-08,0,6,16,24,11,0,34,2,3,1,2,7,1,0,0,0,0,1,0,SP,36,1,1,Single,91,0.065934,0.175824,0.263736,0.120879,0.000000,0.373626,8,11.375000,0.250000,0.375000,1.750,1,3555,1,0.025598,1,0.2,1


In [73]:
os.chdir(r'D:\OneDrive - Northeastern University\Jupyter Notebook\Data Science Projects\CRM-Analysis-for-Marketing-data\marketing data')
df.to_csv("marketing_data_with_features.csv", index = False)